In [1]:
!pip install tensorflow

In [2]:
!pip install tensorflow-datasets

In [3]:
# Operasi Numerik
import numpy as np
# Melatih jaringan saraf
import tensorflow_datasets as tfds
import tensorflow as tf

# Nonaktifkan bar
tfds.disable_progress_bar()

In [5]:
# Membuat grafik dan visualisasi
import matplotlib.pyplot as plt

# Mendifiisikan fungsi plot_graph
def plot_graphs(history, metric):
    # Membuat garis untuk metrik pada setiap epoch pada data pelatihan
    plt.plot(history.history[metric])
    # Membuat garis untuk metrik pada setiap epoch
    plt.plot(history.history['val_'+metric], '')
    # Memberi label pada sumbu x
    plt.xlabel("Epochs")
    # Memberi label pada sumbu y
    plt.ylabel(metric)
    # Menyertakan legenda untuk membedakan antara data pelatihan dan data validasi
    plt.legend([metric, 'val_'+metric])


**Setup input pipeline**

Dataset ulasan film IMDB adalah kumpulan data klasifikasi biner—semua ulasan memiliki sentimen positif atau negatif.

In [6]:
# Memuat dataset IMDb
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
# Memisahkan dataset menjadi dua bagian
train_dataset, test_dataset = dataset['train'], dataset['test']

# Menampilkan spesifikasi elemen dari dataset pelatihan
train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

Awalnya ini mengembalikan dataset (teks, pasangan label):

In [7]:
# Mengambil satu contoh dari dataset pelatihan
for example, label in train_dataset.take(1):
  #  Mencetak teks ulasan dari contoh yang diambil
  print('text: ', example.numpy())
  #  Mencetak label dari contoh yang diambil
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


Berikutnya acak data untuk pelatihan dan membuat kumpulan pasangan (teks, label) ini:

In [8]:
# Ukuran buffer yang digunakan saat melakukan pengacakan
BUFFER_SIZE = 10000
# Ukuran batch yang digunakan saat membentuk batch dari dataset
BATCH_SIZE = 64

# Proses pelatihan
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Mengambil satu batch dari dataset pelatihan
for example, label in train_dataset.take(1):
  # Mencetak tiga contoh teks ulasan dari batch yang diambil
  print('texts: ', example.numpy()[:3])
  print()
  # Mencetak tiga label yang sesuai dengan tiga contoh teks ulasan yang diambil
  print('labels: ', label.numpy()[:3])


texts:  [b"This mindless movie is a piece of crap and boring like the full house repetitions. For all the people who want to see a great, exciting and cool horror movie shouldn't even think about watching this bunch of mindless work. a F- in my opinion. I have one question, what were they thinking? Let's make a list: 1) bad script 2) bad script 3) bad script 4) bad acting 5) bad directing and last but not least a bad script. I mean I am not like grumping about every movie, but I was disappointed when I watched it. This movie should be banned into a box, locked and sunk down into the sea. So please don't do something like this again, please, please, please!!!!"
 b'Some people might consider this movie a piece of artwork - to be able to express your imagination on film in order to create a movie filled with antagonizing pain and death.. I personally think that this movie is a disgust, which should have never been released. This movie is repulsive, illogical and meaningless. Not only is i

**Buat Teks Encoder**

Teks mentah yang dimuat oleh tfds perlu diproses sebelum dapat digunakan dalam model. Cara termudah memproses teks untuk pelatihan adalah menggunakan lapisan TextVectorization. Lapisan ini memiliki banyak kemampuan, namun pada tutorial ini menggunakan perilaku default. Buat lapisan tersebut, dan teruskan teks kumpulan data ke metode .adapt lapisan:

In [9]:
# Mendefinisikan ukuran vokabuler yang akan digunakan
VOCAB_SIZE = 1000
# Mengonversi teks ke dalam representasi numerik
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
# Memetakan fungsi pada setiap contoh dalam dataset
encoder.adapt(train_dataset.map(lambda text, label: text))

Metode .adapt mengatur kosakata lapisan. 20 token pertama dapat dilihat dengan kode berikut. Setelah padding dan token yang tidak diketahui, mereka diurutkan berdasarkan frekuensi:

In [11]:
# Mengembalikan vokabuler dalam bentuk Python list
vocab = np.array(encoder.get_vocabulary())
#  Mengambil 20 kata pertama dari vokabuler
vocab[:20]


array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

Setelah kosakata diatur, lapisan dapat mengkodekan teks ke dalam indeks. Tensor indeks diberi bantalan 0 ke urutan terpanjang dalam batch (kecuali jika Anda menetapkan output_sequence_length tetap):

In [13]:
# Mengubah teks menjadi numerik
encoded_example = encoder(example)[:3].numpy()
encoded_example


array([[ 11,   1,  18, ...,   0,   0,   0],
       [ 47,  83, 227, ...,   0,   0,   0],
       [  1,   1,   1, ...,   0,   0,   0]])

Dengan pengaturan default, prosesnya tidak dapat dibalik sepenuhnya. Ada dua alasan utama untuk itu:
-Nilai default untuk argumen standarisasi preprocessing.TextVectorization adalah "lower_and_strip_punctuation".
-Ukuran kosa kata yang terbatas dan kurangnya fallback berbasis karakter menghasilkan beberapa token yang tidak diketahui.

In [14]:
# Mencetak informasi untuk 3 elemen pertama
for n in range(3):
  # Mencetak teks asli dari contoh pada indeks
  print("Original: ", example[n].numpy())
  # Mencetak teks yang telah diubah ke dalam representasi numerik
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b"This mindless movie is a piece of crap and boring like the full house repetitions. For all the people who want to see a great, exciting and cool horror movie shouldn't even think about watching this bunch of mindless work. a F- in my opinion. I have one question, what were they thinking? Let's make a list: 1) bad script 2) bad script 3) bad script 4) bad acting 5) bad directing and last but not least a bad script. I mean I am not like grumping about every movie, but I was disappointed when I watched it. This movie should be banned into a box, locked and sunk down into the sea. So please don't do something like this again, please, please, please!!!!"
Round-trip:  this [UNK] movie is a piece of crap and boring like the full house [UNK] for all the people who want to see a great [UNK] and cool horror movie [UNK] even think about watching this bunch of [UNK] work a [UNK] in my opinion i have one question what were they thinking lets make a [UNK] 1 bad script 2 bad script 3 bad

**Buat Model**

In [15]:
# Membuat model sequential dengan lapisan-lapisan yang telah didefinisikan
model = tf.keras.Sequential([
    encoder,
    # Mengambil representasi numerik dari 'encoder' dan mengonversinya ke dalam vektor embedding
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    # Memproses urutan teks dalam kedua arah
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    # memproses fitur-fitur yang dihasilkan oleh lapisan LSTM sebelumnya
    tf.keras.layers.Dense(64, activation='relu'),
    # Digunakan sebagai layer output
    tf.keras.layers.Dense(1)
])

In [16]:
# Mencetak apakah setiap lapisan dalam model mendukung masking atau tidak
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


**Penjelasan**

Lapisan pertama tidak mendukung masking, sedangkan pada 4 lapisan berikutnya mendukung

In [17]:
# Mendefinisikan sebuah contoh teks sebagai string
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
# Melakukan prediksi pada teks sample yang awalnya harus dikonversi dulu ke array numpy
predictions = model.predict(np.array([sample_text]))
# Mencetak hasil prediksi
print(predictions[0])

1/1 [==============================] - 4s 4s/step
[0.00036225]


In [18]:
# Membuat sebuah teks panjang yang diulang sebanyak 2000 kali
padding = "the " * 2000
#  Melakukan prediksi pada sample teks, dan padding (the 200 kali)
predictions = model.predict(np.array([sample_text, padding]))
# Mencetak prediksi sentimen
print(predictions[0])

1/1 [==============================] - 0s 363ms/step
[0.00036225]


In [19]:
# Mengonfigurasi model setelah model tersebut didefinisikan
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

**Train Model**

In [20]:
#latih model
#jumlah iterasi melatih model(epoch)=10
#validation_data=test_dataset:data validasi yang digunakan untuk mengukur kinerja model selama pelatihan. Data ini berisi teks ulasan dan labelnya.
#validation_steps=30:jumlah langkah validasi yang akan diambil selama setiap epoch
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset,
validation_steps=30)

Epoch 1/10
391/391 [==============================] - 817s 2s/step - loss: 0.6479 - accuracy: 0.5698 - val_loss: 0.5116 - val_accuracy: 0.7599
Epoch 2/10
391/391 [==============================] - 763s 2s/step - loss: 0.4119 - accuracy: 0.8149 - val_loss: 0.3745 - val_accuracy: 0.8411
Epoch 3/10
391/391 [==============================] - 769s 2s/step - loss: 0.3420 - accuracy: 0.8493 - val_loss: 0.3515 - val_accuracy: 0.8542
Epoch 4/10
391/391 [==============================] - 704s 2s/step - loss: 0.3246 - accuracy: 0.8591 - val_loss: 0.3448 - val_accuracy: 0.8552
Epoch 5/10
391/391 [==============================] - 702s 2s/step - loss: 0.3153 - accuracy: 0.8636 - val_loss: 0.3260 - val_accuracy: 0.8521
Epoch 6/10
391/391 [==============================] - 750s 2s/step - loss: 0.3117 - accuracy: 0.8649 - val_loss: 0.3231 - val_accuracy: 0.8526
Epoch 7/10
391/391 [==============================] - 798s 2s/step - loss: 0.3070 - accuracy: 0.8682 - val_loss: 0.3451 - val_accuracy: 0.8599

In [ ]:
# Mengevaluasi performa model pada dataset pengujian
test_loss, test_acc = model.evaluate(test_dataset)

# Mencetak nilai loss pada dataset pengujian
print('Test Loss:', test_loss)
# Mencetak akurasi pada dataset pengujian
print('Test Accuracy:', test_acc)

 31/391 [=>............................] - ETA: 2:42 - loss: 0.3200 - accuracy: 0.8558

In [ ]:
# Membuat figure untuk grafik dengan ukuran 16x8 inci
plt.figure(figsize=(16, 8))
# Membuat subplot pertama pada gambar
plt.subplot(1, 2, 1)
# Memanggil fungsi 'plot_graphs'
plot_graphs(history, 'accuracy')
# Menetapkan batas sumbu y
plt.ylim(None, 1)
# Membuat subplot kedua pada gambar
plt.subplot(1, 2, 2)
# Memanggil fungsi 'plot_graphs'
plot_graphs(history, 'loss')
# Menetapkan batas sumbu y
plt.ylim(0, None)

**Penjelasan**

Nilai accuracy semakin naik seiring dengan beerjalannya epoch, sedangkan nilai loss / kerugian terus berkurang seiring berjalannya epoch

Accuracy : akurasi model pada data pelatihan selama berjalannya epoch. Jika akurasi pelatihan terus meningkat seiring berjalannya epoch, itu menunjukkan bahwa model belajar dengan baik dari data pelatihan.

Val Accuracy : akurasi model pada data validasi yang merupakan data yang tidak digunakan dalam pelatihan. Jika akurasi validasi naik bersamaan dengan akurasi pelatihan, itu menunjukkan bahwa model dapat menggeneralisasi dengan baik ke data yang belum pernah dilihat sebelumnya.

Loss: Nilai kehilangan model pada data pelatihan seiring berjalannya epoch. Jika nilai loss pelatihan terus menurun seiring dengan peningkatan jumlah epoch, itu menunjukkan bahwa model terus mempelajari data pelatihan.

Val Loss: Validation loss adalah nilai kehilangan model pada data validasi yang merupakan data yang tidak digunakan dalam pelatihan. Jika nilai validation loss juga menurun seiring dengan peningkatan epoch, itu menunjukkan bahwa model dapat melakukan generalisasi dengan baik pada data yang belum pernah dilihat sebelumnya.

In [ ]:
# Mendefinisikan kalimat baru sebagai string
sample_text = ('The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.')
# Menggunakan model untuk melakukan prediksi
predictions = model.predict(np.array([sample_text]))

**Penjelasan**

Hasil prediksi "1/1" menunjukkan bahwa model memprediksi teks tersebut sebagai sentimen positif dan prediksinya benar.


**Stack two or more LSTM layers**

In [ ]:
# Membuat model sequential dengan lapisan yang telah didefinisikan
model = tf.keras.Sequential([
    encoder,
     tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
     tf.keras.layers.Dense(64, activation='relu'),
     tf.keras.layers.Dropout(0.5),
     tf.keras.layers.Dense(1)
])

In [ ]:
# Mengonfigurasi model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
 optimizer=tf.keras.optimizers.Adam(1e-4),
 metrics=['accuracy'])

In [ ]:
# Melatih model dengan iterasi epoch 10
history = model.fit(train_dataset, epochs=10,
 validation_data=test_dataset,
 validation_steps=30)

In [ ]:
# Melakukan evaluasi akurasi dan loss pada model
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
# Memprediksi sentimen pada satu contoh teks
sample_text = ('The movie was not good. The animation and the graphics were terrible. I would not recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions)

**Penjelasan**

Penilaian model rendah terhadap teks

In [ ]:
# Mencetak plot akurasi dan loss
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')

**Penjelasan**

Accuracy : akurasi model pada data pelatihan selama berjalannya epoch. Jika akurasi pelatihan terus meningkat seiring berjalannya epoch, itu menunjukkan bahwa model belajar dengan baik dari data pelatihan.

Val Accuracy : akurasi model pada data validasi yang merupakan data yang tidak digunakan dalam pelatihan. Jika akurasi validasi naik bersamaan dengan akurasi pelatihan, itu menunjukkan bahwa model dapat menggeneralisasi dengan baik ke data yang belum pernah dilihat sebelumnya.

Loss: Nilai kehilangan model pada data pelatihan seiring berjalannya epoch. Jika nilai loss pelatihan terus menurun seiring dengan peningkatan jumlah epoch, itu menunjukkan bahwa model terus mempelajari data pelatihan.

Val Loss: Validation loss adalah nilai kehilangan model pada data validasi yang merupakan data yang tidak digunakan dalam pelatihan. Jika nilai validation loss juga menurun seiring dengan peningkatan epoch, itu menunjukkan bahwa model dapat melakukan generalisasi dengan baik pada data yang belum pernah dilihat sebelumnya.